In [7]:
from typing import List
from collections import namedtuple
import hashlib

Node = namedtuple('nodes', ['id'])
Edge = namedtuple('edge', ['id', 'src', 'dst', 'length'])

In [10]:
with open('../data/shortest_route/citymapper-coding-test-graph.dat', 'r') as f:
    data = f.readlines()

In [11]:
nodes = []
edges = []

# Number of vertices
N = int(data[0].strip("\n"))
for i in range(1, N+1):
    nodes.append(Node(int(data[i].strip("\n"))))

# Number of edges
E = int(data[N+1])
for i in range(N+2, E+1):
    src, dst, length = data[i].strip("\n").split(" ")
    e_id = hashlib.sha256("{}_{}".format(src,dst).encode()).hexdigest()
    edges.append(Edge(e_id, int(src), int(dst), int(length)))

In [29]:

class CityMapperGraph:
    
    def __init__(self, nodes: List[Node], edges: List[Edge]):
        self.nodes = nodes
        self.edges = edges
        self.adjacency = {}
        
    def _populate(self):
        for e in self.edges:
            # Populate adjacency list for source -> destination
            if e.src in self.adjacency:
                self.adjacency[e.src].update([e.dst])
            else:
                self.adjacency[e.src] = set([e.dst])
            # Populate adjacency list for destination -> source
            if e.dst in self.adjacency:
                self.adjacency[e.dst].update([e.src])
            else:
                self.adjacency[e.dst] = set([e.src])
            

In [30]:
graph = CityMapperGraph(nodes=nodes, edges=edges)
graph._populate()

In [31]:
len(graph.adjacency)

2516